In [ ]:
from google.colab import drive,auth
drive.mount('/content/drive')
auth.authenticate_user()

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=b13d13c87d5a30b4b8737f3e2a8b004b5033b822f4b98d46f7b29d22295fd006
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import re, string
from bs4 import BeautifulSoup
import emoji
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from textblob import TextBlob
from textblob import Word
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))
vectorizer = CountVectorizer(ngram_range=(1, 1))
estimator = DecisionTreeClassifier(max_depth=3)

In [ ]:
contraction_map={"ain't":'is not',"aren't":'are not',"can't":'can not','cause':'because',"could've":'could have',"couldn't":'could not',"couldn't've":'could not have',"didn't":'did not',"doesn't":'does not',"don't":'do not',"hadn't":'had not',"hadn't've":'had not have',"hasn't":'has not',"haven't":'have not',"he'd":'he would',"he'd've":'he would have',"he'll":'he will',"he'll've":'he will have',"he's":'he is',"how'd":'how did',"how'd've":'how did have',"how'll":'how will',"how's":'how is',"I'd":'I would',"I'd've":'I would have',"I'll":'I will',"I'll've":'I will have',"I'm":'I am',"I've":'I have',"i'd":'i would',"i'd've":'i would have',"i'll":'i will',"i'll've":'i will have',"i'm":'i am',"i've":'i have',"isn't":'is not',"it'd":'it would',"it'd've":'it would have',"it'll":'it will',"it'll've":'it will have',"it's":'it is',"let's":'let us',"ma'am":'madam',"mayn't":'may not',"might've":'might have',"mightn't":'might not',"mightn't've":'might not have',"must've":'might have',"mustn't":'must not',"mustn't've":'must not have',"needn't":'need not',"needn't've":'need not have',"o'clock":'of the clock',"oughtn't":'ought not',"oughtn't've":'ought not have',"shan't":'shall not',"shall'n't":'shall not',"shan't've":'shall not have',"she'd":'she would',"she'd've":'she would have',"she'll":'she will',"she'll've":'she will have',"she's":'she is',"should've":'should have',"shouldn't":'should not',"shouldn't've":'should not have',"so've":'so have',"so's":'so as',"that'd":'that would',"that'd've":'that would have',"that's":'that is',"there'd":'there would',"there'd've":'there would have',"there's":'there is',"they'd":'they would',"they'd've":'they would have',"they'll":'they will',"they'll've":'they will have',"they're":'they are',"they've":'they have',"to've":'to have',"wasn't":'was not',"we'd":'we would',"we'd've":'we would have',"we'll":'we will',"we'll've":'we will have',"we're":'we are',"weren't":'were not',"what'll":'what will',"what'll've":'what will have',"what're":'what are',"what's":'what is',"what've":'what have',"when's":'when is',"when've":'when have',"where'd":'where did',"where's":'where is',"where've":'where have',"who'll":'who will',"who'll've":'who will have',"who's":'who is',"who've":'who have',"why's":'why is',"why've":'why have',"will've":'will have',"won't":'will not',"will't've":'will not have',"would've":'would have',"would't":'would not',"would't've":'would not have',"y'all":'you all',"y'all'd":'you all would',"y'all'd've":'you all would have',"y'all're":'you all are',"y'all've":'you have all',"you'd":'you would',"you'd've":'you would have',"you'll":'you will',"you'll've":'you will have',"you're":'you are',"you've":'you have'}
eng_words = set(nltk.corpus.words.words())

In [ ]:
def expand_contractions(sent, mapping):
    pattern = re.compile('({})'.format('|'.join(mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_map(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expansion = mapping.get(match) if mapping.get(match) else mapping.get(match.lower())
        expansion = first_char + expansion[1:]
        return expansion
    expand_sent = pattern.sub(expand_map, sent)
    return expand_sent

def split_hashtag(text):
    words = text.split(" ")
    text = ''
    for word in words:
        if(len(word)>0):
            if word[0]=='#':
                tmp=''
                pre=''
                for ch in word:
                    if ch!='#':
                        if (ch<='Z' and ch>='A') and not(pre<='Z' and pre>='A'):
                            tmp+=' '
                        tmp+=ch
                    pre=ch
                word = tmp
        text=text+' '+word
    return text

def remove_singular(text):
    words = text.split(" ")
    text = ''
    for word in words:
        if len(word)>1:
            text=text+' '+word
    return text

def replace_emoji(text):
    text = emoji.demojize(text)
    return text

def alpha_only(text):
    tmp=''
    for ch in text:
        if ch>='a' and ch<='z':
            tmp+=ch
        else:
            tmp+=' '
    return tmp

def english_only(text):
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in eng_words or not w.isalpha())
    return text

def text_cleaning(text):
    text = BeautifulSoup(text).get_text()
    text = re.sub(r"(?:https?\://)\S+", "", text)
    text=re.sub(r"(?:\@)\S+","user",text)
    text = re.sub('\n','',text)
    text = re.sub('\t','',text)
    text=split_hashtag(text)
    text=replace_emoji(text)
    text=text.lower()
    text = text.replace("’","'")
    text=expand_contractions(text,contraction_map)
    text=alpha_only(text)
    text = ' '.join(text.split())
    pbar.update(1)
    return text;

In [ ]:
def load_tf_dataset(name,dir=None):
    if dir==None:
        dataset = load_dataset(name)
    else:
        dataset = load_dataset(name,dir)
    return dataset

def dataset_to_train_test(dataset):
    train_df = pd.DataFrame(dataset["train"])
    test_df = pd.DataFrame(dataset["test"])
    return train_df, test_df

def load_csv_dataset(dir,filename):
    df = pd.read_csv(dir+filename)
    return df

def load_xlxs_dataset(dir,filename):
    df = pd.read_excel(dir+filename,index_col=None)
    return df

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

def df_cleaner(df):
    df['text'] = df['text'].apply(text_cleaning)
    df['text'] = df['text'].apply(lambda x: x.split())
    df['text'] = df['text'].apply(remove_stopwords)
    df['text'] = df['text'].apply(lambda x: ' '.join(x))
    return df

def split_dataset(df):
    X = vectorizer.fit_transform(df['text'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def assign_dataset(train_df,test_df):
    X_train = vectorizer.fit_transform(train_df['text'])
    X_test = vectorizer.transform(test_df['text'])
    y_train = train_df['label']
    y_test = test_df['label']
    return X_train, X_test, y_train, y_test

def pred_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    LR_pred = model.predict(X_test)
    return LR_pred

def report_gen(y_test, y_pred):
    report = classification_report(y_test,y_pred)
    return report

In [ ]:
def vader_sentiment(score):
    threshold = 0.1
    if score['compound'] > threshold:
        label = 2
    elif score['compound'] < -threshold:
        label = 0
    else: label = 1
    return label

def pred_VEDER(model, X, y):
    y_pred = X.apply(lambda x: model.polarity_scores(x))
    y_pred = y_pred.apply(vader_sentiment)
    return y_pred

In [ ]:
LR = LogisticRegression()
MNB = MultinomialNB()
BAGG = BaggingClassifier(estimator=estimator, n_estimators=10)
ADA = BaggingClassifier(estimator=estimator, n_estimators=50)
GRAD = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
VADER = SentimentIntensityAnalyzer()
SGD = SGDClassifier()

In [ ]:
def get_all_report(X_train, X_test, y_train, y_test, test_df):
    print("Accuracy report of LR:")
    y_pred = pred_model(LR, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of MNB:")
    y_pred = pred_model(MNB, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of BAGG:")
    y_pred = pred_model(BAGG, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of ADA:")
    y_pred = pred_model(ADA, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of GRAD:")
    y_pred = pred_model(GRAD, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of SGD:")
    y_pred = pred_model(SGD, X_train, X_test, y_train, y_test)
    report = report_gen(y_test, y_pred)
    print(report)
    print("Accuracy report of VADER:")
    y_pred = pred_VEDER(VADER, test_df['text'], test_df['label'])
    report = report_gen(test_df['label'], y_pred)
    print(report)

In [ ]:
def fix_prob_140(df):
    df = df[['text','sentiment']]
    df = df.dropna()
    df=df.rename(columns={"text": "text", "sentiment": "label"})
    df=df.loc[df['label'] != 2]
    df['label'] = df['label'].apply(lambda x: 0 if x==0 else 2)
    return df

dataset = load_tf_dataset("sentiment140")
train_df,test_df = dataset_to_train_test(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
limit = len(train_df)
pbar = tqdm(total=limit, position=0, leave=True)
train_df = df_cleaner(train_df)
pbar.close()
test_df = df_cleaner(test_df)

train_df = fix_prob_140(train_df)
test_df = fix_prob_140(test_df)

100%|██████████| 1600000/1600000 [15:08<00:00, 1760.54it/s]


In [ ]:
src_path="/content/drive/MyDrive/Research/SentimentAnalysis/ExternalDatasets/"
train_ck="sentiment140_after_cleaning_train.csv"
test_ck="sentiment140_after_cleaning_test.csv"
#train_df.to_csv(src_path+train_ck,index=False)
#test_df.to_csv(src_path+test_ck,index=False)
train_df = pd.read_csv(src_path+train_ck)
test_df = pd.read_csv(src_path+test_ck)
train_df

,text,label
0,user awww bummer shoulda got david carr third day,0
1,upset update facebook texting might cry result...,0
2,user dived many times ball managed save rest g...,0
3,whole body feels itchy like fire,0
4,user behaving mad bececause see,0
...,...,...
1599995,woke school best feeling ever,2
1599996,thewdb com cool hear old walt interviews,2
1599997,ready mojo makeover ask details,2
1599998,happy th birthday boo alll time tupac amaru sh...,2


In [ ]:
test_df

,text,label
0,user loooooooovvvvvveee kindle dx cool fantast...,2
1,reading kindle love lee childs good read,2
2,ok first assesment kindle fucking rocks,2
3,user love kindle mine months never looked back...,2
4,user fair enough kindle think perfect,2
...,...,...
354,using latex lot typeset mathematics looks hideous,2
355,note hate word hate pages hate latex said hate...,0
356,ahhh back real text editing environment latex,2
357,trouble iran see hmm iran iran far away flocko...,0


In [ ]:
train_df = train_df.dropna()
test_df = test_df.dropna()
labels = train_df['label'].value_counts().sort_index().reset_index(name='counts')
display(labels)
labels = test_df['label'].value_counts().sort_index().reset_index(name='counts')
display(labels)

,index,counts
0,0,799710
1,2,799713


,index,counts
0,0,177
1,2,182


In [ ]:
X_train, X_test, y_train, y_test = assign_dataset(train_df, test_df)

In [ ]:
get_all_report(X_train, X_test, y_train, y_test, test_df)

Accuracy report of LR:
              precision    recall  f1-score   support

           0       0.83      0.78      0.80       177
           2       0.80      0.84      0.82       182

    accuracy                           0.81       359
   macro avg       0.81      0.81      0.81       359
weighted avg       0.81      0.81      0.81       359

Accuracy report of MNB:
              precision    recall  f1-score   support

           0       0.81      0.79      0.80       177
           2       0.80      0.82      0.81       182

    accuracy                           0.81       359
   macro avg       0.81      0.80      0.80       359
weighted avg       0.81      0.81      0.80       359

Accuracy report of BAGG:
              precision    recall  f1-score   support

           0       0.54      0.80      0.65       177
           2       0.64      0.35      0.45       182

    accuracy                           0.57       359
   macro avg       0.59      0.57      0.55       359
we

In [ ]:
import pickle

In [ ]:
src_path="/content/drive/MyDrive/Research/SentimentAnalysis/ExternalDatasets/"

file = open(src_path+'vectoriser.pickle','wb')
pickle.dump(vectorizer, file)
file.close()

file = open(src_path+'LR.pickle','wb')
pickle.dump(LR, file)
file.close()

file = open(src_path+'MNB.pickle','wb')
pickle.dump(MNB, file)
file.close()

file = open(src_path+'BAGG.pickle','wb')
pickle.dump(BAGG, file)
file.close()

file = open(src_path+'ADA.pickle','wb')
pickle.dump(ADA, file)
file.close()

file = open(src_path+'GRAD.pickle','wb')
pickle.dump(GRAD, file)
file.close()

file = open(src_path+'SGD.pickle','wb')
pickle.dump(SGD, file)
file.close()

